In [ ]:
pip install fuzzywuzzy


In [ ]:
#pip install pandas numpy matplotlib statsmodels
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt

***1.Import dataset***

In [ ]:
file_path =r'C:\Users\yumi7517\OneDrive - University of Otago\Desktop\FINC503\A1 material-python\DATA for CLASS\NZDATA-FORCALCULATINGAMIHUD.csv'
df = pd.read_csv(file_path)
df.head(5)


***2.Data Cleaning***


In [ ]:
df['date'] = pd.to_datetime(df['Date'])
df['month'] = df['date'].dt.to_period('M')
df['RET_INDEX'] = pd.to_numeric(df['TOT RETURN IND'], errors='coerce')
df['PRC'] = pd.to_numeric(df['UNADJUSTED PRICE'], errors='coerce')
df['VOL'] = pd.to_numeric(df['TURNOVER BY VOLUME'], errors='coerce')

#datastream VOL was divided 1000, therefore we should multuple 1000
df['VOL']=df['VOL']*1000

#keep stocks that daily Volume>20000, to ensure we exclude all the stocks will extreme low liqudity
df = df[df['VOL'] > 20000]
df = df[df['PRC'] > 1]

#df.head(5)
print(df)


***3.covert TOT RETURN IND*** TO SIMPLE REUTRN
* Since the data is from data stream, total return index (TOT RETURN IND) is the return index inlcduing dividends.
* To convert simple return, ***((TOT RETURN IND )t  - (TOT RETURN IND )t-1)-1***

In [ ]:
df['RET'] = df['TOT RETURN IND'].pct_change()
df.head()

***4.Create Amihud Illiquidity for each stock each day***
Amihud illiquidity= ((absolute value of Return)/(vol*PRC))*10^6


high amihud value --->illiquidity
low amihud value---->liquidity

In [ ]:
df['DOLLAR_VOL'] = df['VOL'] * df['PRC'].abs()
df['amihud_daily'] = df['RET'].abs() / df['DOLLAR_VOL']*(10**6)
df.head()

***5.Groupby and obtain monthly Amihud Illiquidity for each stock***

In [ ]:
df['month'] = df['date'].dt.to_period('M')


df = df[df['DOLLAR_VOL'] > 0]###only keep dollar volume rows   abs_ret_div_dollarvol

# calculate Amihud illiquidity group by month and stock name
amihud_illiq = df.groupby(['Stock_Name', 'month'])['amihud_daily'].mean().reset_index()
amihud_illiq.rename(columns={'amihud_daily': 'Amihud_Illiquidity'}, inplace=True)
#amihud_illiq['Amihud_Transform']=amihud_illiq['Amihud_Illiquidity']
amihud_illiq.head(5)

***6.Create quintiles for Each year based on Amihud Illiquidity***
for each year, we divided sotck for 5 quintiles, top 20% stocks month with lowest Amihud Illiquidity at Q1, 
20% stock month with second lowest Amihud Illiquidity at Q2, etc.

In [ ]:
amihud_illiq['illiquidity_quintile'] = amihud_illiq.groupby('month')['Amihud_Illiquidity']\
    .transform(lambda x: pd.qcut(x, 5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5']))
##lamba x is a temporary def, x here is all Amihud illuidity at a month. qcut divide the input x into 5 quintiles.

amihud_illiq.head(5)
#merged_df_Amihud_RET.tail(5)

***7.Calculate each quintile's average Amihud illiquity***

In [ ]:
from scipy.stats.mstats import winsorize

#winsorize
amihud_values = amihud_illiq['Amihud_Illiquidity'].values
amihud_winsorized = winsorize(amihud_values, limits=[0.01, 0.01])
amihud_illiq['Amihud_Illiquidity_winsorized'] = amihud_winsorized

#calculate each quintile's average Amihud_Illiquidity
avg_illiquidity_by_quintile = amihud_illiq.groupby('illiquidity_quintile')['Amihud_Illiquidity_winsorized'].mean()

print(avg_illiquidity_by_quintile)

****7.Import Each firm's monthly return****

In [ ]:
#firstly, show current dataframe
amihud_illiq.head()

In [ ]:
import numpy as np

file_path =r'C:\Users\yumi7517\OneDrive - University of Otago\Desktop\FINC503\A1 material-python\RAW NZ-DATA FOR A1\final\DATA for CLASS\NZDATA-MonthData.csv'
df_MRET = pd.read_csv(file_path)

# Parse date and set month
df_MRET['Date'] = pd.to_datetime(df_MRET['Date'], dayfirst=True)
df_MRET['month'] = df_MRET['Date'].dt.to_period('M')

# Convert TOT RETURN IND to numeric (cleaning step)
df_MRET['TOT RETURN IND'] = pd.to_numeric(df_MRET['TOT RETURN IND'], errors='coerce')
df_MRET['MARKET CAPITALIZATION'] = pd.to_numeric(df_MRET['MARKET CAPITALIZATION'], errors='coerce')

# Drop rows where TOT RETURN IND is missing
df_MRET = df_MRET.dropna(subset=['TOT RETURN IND'])

# Sort and compute returns
df_MRET = df_MRET.sort_values(['Stock_Name', 'Date'])

df_MRET['monthly_return'] = (
    df_MRET
    .groupby('Stock_Name')['TOT RETURN IND']
    .pct_change()
)

df_MRET['monthly_log_return'] = np.log(1 + df_MRET['monthly_return'])

df_MRET.head()



In [ ]:
df_MRET.to_csv(r'C:\Users\yumi7517\OneDrive - University of Otago\Desktop\FINC503\A1 material-python\RAW NZ-DATA FOR A1\final\DATA for CLASS\test\mret222.csv', index=False)

***8.merge Monthly_log_return to Amihud and Quintile, groupby month and permno***

In [ ]:
from fuzzywuzzy import process
import pandas as pd
from pandas.tseries.offsets import MonthEnd


###we are going to use fuzzy matching in this step. we are trying to merge two files, but the key is not 100% same.
##for example, stock name '2 cheap cars' and '2 cheap cars group'
##therefore, Fuzzy match on company names

# ensure 'month' is datetime
if pd.api.types.is_period_dtype(df_MRET['month']):
    df_MRET['month'] = df_MRET['month'].dt.to_timestamp()
else:
    df_MRET['month'] = pd.to_datetime(df_MRET['month'])

if pd.api.types.is_period_dtype(amihud_illiq['month']):
    amihud_illiq['month'] = amihud_illiq['month'].dt.to_timestamp()
else:
    amihud_illiq['month'] = pd.to_datetime(amihud_illiq['month'])

# align date
df_MRET['month'] = df_MRET['month'] + MonthEnd(0)
amihud_illiq['month'] = amihud_illiq['month'] + MonthEnd(0)


# Step 2: Fuzzy mapping
stock_names_MRET = df_MRET['Stock_Name'].unique()
stock_names_ill = amihud_illiq['Stock_Name'].unique()

name_map = {}
for name in stock_names_ill:
    match, score = process.extractOne(name, stock_names_MRET)
    if score >= 90:
        name_map[name] = match

amihud_illiq['Stock_Name_matched'] = amihud_illiq['Stock_Name'].map(name_map)

# Step 3: Merge
merged_df_Amihud_MRET = pd.merge(
    df_MRET,
    amihud_illiq,
    left_on=['month', 'Stock_Name'],
    right_on=['month', 'Stock_Name_matched'],
    how='inner'
)

merged_df_Amihud_MRET['month'] = merged_df_Amihud_MRET['Date'].dt.to_period('M')

print(merged_df_Amihud_MRET)

In [ ]:
merged_df_Amihud_MRET.to_csv(r'C:\Users\yumi7517\OneDrive - University of Otago\Desktop\FINC503\A1 material-python\RAW NZ-DATA FOR A1\final\DATA for CLASS\test\mret.csv', index=False)

***9.import ff3 and CAPM factors***

In [ ]:
file_path =r'C:\Users\yumi7517\OneDrive - University of Otago\Desktop\FINC503\A1 material-python\RAW NZ-DATA FOR A1\final\DATA for CLASS\NZDATA-AlphaFactors.xlsx'
df_factor = pd.read_excel(file_path)
df_factor['date'] = pd.to_datetime(df_factor['Date'])
df_factor['month'] = df_factor['date'].dt.to_period('M')
df_factor.head(5)

***10.merge month ff3 and CAPM factors Amihud and Quintile, groupby month***

In [ ]:

merged_df_Amihud_MRET_ff3 = pd.merge(df_factor, merged_df_Amihud_MRET, on=[ 'month'], how='inner')

#merged_df_Amihud_MRET_ff3.head(5)
print(merged_df_Amihud_MRET_ff3)


***11.This is our data set for final analysis, we can save this to csv***

In [ ]:
merged_df_Amihud_MRET_ff3.to_csv(r'C:\Users\yumi7517\OneDrive - University of Otago\Desktop\FINC503\A1 material-python\RAW NZ-DATA FOR A1\final\DATA for CLASS\test\merged_df_Amihud_MRET_ff3.csv', index=False)

***12. calculate monthly equal-weighted return***
we already get the monthly return for each stock, so we can just calculate equal-weighted return for each month
Equal-weighted return is the average return of a portfolio where each stock is assigned the same weight, regardless of its market capitalization.

equal weighted return is for the market, not the single stocks。

In [ ]:
import pandas as pd


file_path =r'C:\Users\yumi7517\OneDrive - University of Otago\Desktop\FINC503\A1 material-python\RAW NZ-DATA FOR A1\final\DATA for CLASS\test\merged_df_Amihud_MRET_ff3.csv'
df_all = pd.read_csv(file_path)

# each month equal-weighted return
equal_weighted_returns = (
    df_all.groupby("month")["monthly_return"]
    .mean()
    .reset_index()
    .rename(columns={"monthly_return": "equal_weighted_return"})
)


print(equal_weighted_returns.head())

***13. summary statistics for monthly equal-weighted return***

In [ ]:
equal_weighted_returns.describe()

***14. eqch quintile equal weighted return***

In [ ]:
equal_weighted_returns_q = (
    df_all.groupby("illiquidity_quintile")["monthly_return"]
    .mean()
    .reset_index()
    .rename(columns={"monthly_return": "equal_weighted_return_q"})
)
print(equal_weighted_returns_q)

***15.calculate CAPM alpha for each stock each month***
R_it - R_ft = α_i + β_i (R_mt - R_ft) + ε_t

In [ ]:
import pandas as pd
import statsmodels.api as sm

# ---------- 1. generate「each month × each quintile」equal weight return ----------
port_rets = (
    df_all
    .groupby(["month", "illiquidity_quintile"])["monthly_return"]
    .mean()
    .reset_index()
    .rename(columns={"monthly_return": "port_return"})
)

# ---------- 2. merge to FF3 factors ----------
# 提取当月的 Mkt-RF 和 RF，每月一行
factors = df_all[["month", "Mkt-RF", "RF"]].drop_duplicates()
factors[["Mkt-RF", "RF"]] = factors[["Mkt-RF", "RF"]] / 100  # 转换成小数形式

# 合并因子数据
port_rets = port_rets.merge(factors, on="month", how="left")

# ---------- 3. compute contemporaneous excess return ----------
port_rets["excess_return"] = port_rets["port_return"] - port_rets["RF"]

# ---------- 4. generate t+1 excess return ----------
# 按照每个组合排序时间序列，shift(-1) 得到 t+1 的收益
port_rets = port_rets.sort_values(["illiquidity_quintile", "month"])
port_rets["excess_return_t1"] = (
    port_rets.groupby("illiquidity_quintile")["excess_return"]
    .shift(-1)
)

# ---------- 5. regression: using t illiquidity to predict t+1 excess return ----------
results = []

for q in port_rets["illiquidity_quintile"].unique():
    sub = port_rets[port_rets["illiquidity_quintile"] == q].sort_values("month")

    # 删除最后一个月（因为shift后变成NaN）
    sub = sub.dropna(subset=["excess_return_t1"])

    X = sm.add_constant(sub["Mkt-RF"])  # 控制市场因子
    y = sub["excess_return_t1"]         # 被解释变量是 t+1 超额收益

    res = sm.OLS(y, X).fit()

    results.append({
        "quintile"     : q,
        "alpha"        : res.params["const"],
        "alpha_tstat"  : res.tvalues["const"],
        "beta"         : res.params["Mkt-RF"],
        "n_obs"        : len(sub)
    })

# ---------- 6. output summary ----------
alpha_df = pd.DataFrame(results).sort_values("quintile")
print(alpha_df)


***15.CAPM difference between bottom and top quintile***

we are testing, if the excess return most liquidity portfolio  - the least liquidity portfolio significant.


In [ ]:
import pandas as pd
import statsmodels.api as sm


import pandas as pd
import statsmodels.api as sm

#generate「each month × each quintile」ew return
port_rets = (
    df_all
    .groupby(["month", "illiquidity_quintile"])["monthly_return"]
    .mean()
    .reset_index()
    .rename(columns={"monthly_return": "port_return"})
)

#  convert to wide table 
wide = port_rets.pivot(index="month",
                       columns="illiquidity_quintile",
                       values="port_return")

wide["long_short"] = wide["Q5"] - wide["Q1"]      

# 3. merge ff3 factors 
factors = df_all[["month", "Mkt-RF", "RF"]].drop_duplicates()
ls_df   = wide.reset_index().merge(factors, on="month", how="left")



ls_df["excess_ls"] = ls_df["long_short"] - ls_df["RF"]

# only keep non-missing
reg_df = ls_df[["excess_ls", "Mkt-RF"]].dropna()

# 3. construct x, y
X = sm.add_constant(reg_df["Mkt-RF"])
y = reg_df["excess_ls"]
#R_it - R_ft = α_i + β_i (R_mt - R_ft) + ε_t

# 4. 
model = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags": 12})

print(model.summary())         # 现在就不会报错了


***16.calculate ff3 alpha for each stock each month***
(R_it - R_ft) = α_i + β_i (R_mt - R_ft) + s_i · SMB_t + h_i · HML_t + ε_t

In [ ]:
import pandas as pd
import statsmodels.api as sm


port_rets = (
    df_all
    .groupby(["month", "illiquidity_quintile"])["monthly_return"]
    .mean()
    .reset_index()
    .rename(columns={"monthly_return": "port_return"})
)

#merge ff3 factors
factors = df_all[["month", "Mkt-RF", "RF", "smb", "hml"]].drop_duplicates()
factors[["Mkt-RF", "RF", "smb", "hml"]] = factors[["Mkt-RF", "RF", "smb", "hml"]] / 100  # 转成小数

port_rets = port_rets.merge(factors, on="month", how="left")

# calculate excess return
port_rets["excess_return"] = port_rets["port_return"] - port_rets["RF"]

# t+1 port terurn
port_rets = port_rets.sort_values(["illiquidity_quintile", "month"])
port_rets["excess_return_t1"] = (
    port_rets.groupby("illiquidity_quintile")["excess_return"]
    .shift(-1)
)

# ff3 regression
results = []

for q in port_rets["illiquidity_quintile"].unique():
    sub = port_rets[port_rets["illiquidity_quintile"] == q].sort_values("month")

    sub = sub.dropna(subset=["excess_return_t1"])

    X = sub[["Mkt-RF", "smb", "hml"]]
    X = sm.add_constant(X)
    y = sub["excess_return_t1"]

    res = sm.OLS(y, X).fit()

    results.append({
        "quintile"    : q,
        "ff3_alpha"   : res.params["const"],  
        "alpha_tstat" : res.tvalues["const"],
        "beta_mkt"    : res.params["Mkt-RF"],
        "beta_smb"    : res.params["smb"],
        "beta_hml"    : res.params["hml"],
        "n_obs"       : len(sub)
    })

# output
ff3_alpha_df = pd.DataFrame(results).sort_values("quintile")
print(ff3_alpha_df)


***17.difference between bottom and top quintile**

In [ ]:
import pandas as pd
import statsmodels.api as sm

# ---------- 1. 每月 × 每个quintile 的组合收益 ----------
port_rets = (
    df_all
    .groupby(["month", "illiquidity_quintile"])["monthly_return"]
    .mean()
    .reset_index()
    .rename(columns={"monthly_return": "port_return"})
)

# ---------- 2. 转成宽表，构造 Q5-Q1 long-short 组合 ----------
wide = port_rets.pivot(index="month",
                       columns="illiquidity_quintile",
                       values="port_return")

wide["long_short"] = wide["Q5"] - wide["Q1"]

# ---------- 3. 合并 FF3 因子 ----------
factors = df_all[["month", "Mkt-RF", "RF", "smb", "hml"]].drop_duplicates()
factors[["Mkt-RF", "RF", "smb", "hml"]] = factors[["Mkt-RF", "RF", "smb", "hml"]] / 100  # 转换为小数

ls_df = wide.reset_index().merge(factors, on="month", how="left")

# ---------- 4. 计算超额收益 ----------
ls_df["excess_ls"] = ls_df["long_short"] - ls_df["RF"]

# ---------- 5. 构造回归样本 ----------
reg_df = ls_df[["excess_ls", "Mkt-RF", "smb", "hml"]].dropna()

X = sm.add_constant(reg_df[["Mkt-RF", "smb", "hml"]])
y = reg_df["excess_ls"]

# ---------- 6. FF3 回归 + HAC 标准误 ----------
model = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags": 12})

# ---------- 7. 输出结果 ----------
print(model.summary())


###Figure 1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ---- 1. Load & clean basic columns ----
path = r'C:\Users\yumi7517\OneDrive - University of Otago\Desktop\FINC503\A1 material-python\RAW NZ-DATA FOR A1\final\DATA for CLASS\test\merged_df_Amihud_MRET_ff3.csv'
df = pd.read_csv(path)


df['month'] = pd.to_datetime(df['month']) + pd.offsets.MonthEnd(0)

# convert market return to decimal
if df['Mkt-RF'].abs().max() > 1:
    df[['Mkt-RF', 'RF']] = df[['Mkt-RF', 'RF']] / 100

df['Mkt'] = df['Mkt-RF'] + df['RF']

# ---- 2. Equal-weighted return by quintile ----
port_mean = (
    df.groupby(['month', 'illiquidity_quintile'])['monthly_return']
      .mean()
      .unstack()
      .sort_index()
)

#  Q1、Q5、Q5-Q1 and Market
q1 = port_mean.get('Q1')
q5 = port_mean.get('Q5')
long_short = q5 - q1
mkt = df.groupby('month')['Mkt'].mean().reindex(port_mean.index)

# Convert to cumulative log return 
def to_cumulative_log_return(series):
    return np.log1p(series).cumsum()  # log(1 + r)

cum_q1 = to_cumulative_log_return(q1)
cum_q5 = to_cumulative_log_return(q5)
cum_ls = to_cumulative_log_return(long_short)
cum_mkt = to_cumulative_log_return(mkt)

# Plot cumulative returns 
plt.figure(figsize=(11, 6))
plt.plot(cum_q5.index, cum_q5, label='Q5 (Most Illiquid)')
plt.plot(cum_q1.index, cum_q1, label='Q1 (Most Liquid)')
plt.plot(cum_ls.index, cum_ls, label='Long‑Short (Q5 – Q1)')
plt.plot(cum_mkt.index, cum_mkt, label='Market')

plt.title('Cumulative Log Returns: Liquidity Quintiles vs. Market')
plt.xlabel('Month')
plt.ylabel('Cumulative Log Return')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
